# Clean Data
## Introduction
To measure the quality of the air we can use Sulfur Monoxide (SO), Nitrogen Dioxide (NO2), Carbon Dioxide (CO2), Ozone (O3) or Total Suspended Particules. The data obtained by OpenAQ is provided by the different sensors around the world. Nevertheless, we will focus on our country during the last year. We consider using 2020 because it is a year that we have a pandemic situation and we would know what happens during the lockdown etc.

## Get started

>Note: this notebook uses python 3 as kernel

This notebook assumes tha data is already downloaded and stored at ``../data/raw``

if not, execute:

``python ../src/data/get_data.py``

## Read Files

We will be using, at first instance, the following python modules:
- ``os`` python built-in package to deal with directories/files
- ``json`` python built-in package to code/encode JSON data
- ``pandas`` to read and process the data
- ``display`` to visualize the data
- ``pickle`` to keep stored the read and processed data as a byte stream to save loading times

In [1]:
import pandas as pd
import os
import json
from pandas_profiling import ProfileReport
from IPython.display import display
import pickle as pkl

# global vars
rel_path="../data/raw/"
clean_path="../data/clean/"
pkl_path="../data/interim/"
rep_path="../reports/"

pkl_parameters = pkl_path + "parameters.pkl"
pkl_countries = pkl_path + "countries.pkl"
pkl_measurements = pkl_path + "measurements.pkl"
pkl_locations = pkl_path + "locations.pkl"

os.makedirs(pkl_path, exist_ok=True)
os.makedirs(rep_path, exist_ok=True)

In [2]:
def read_parameters():
    _json = json.load(open(rel_path + 'parameters.json'))
    data = pd.DataFrame(_json["results"]).set_index('id')
    data.to_pickle(pkl_parameters, compression='infer', protocol=-1)
    return data

parameters = pkl.load(open(pkl_parameters, 'rb')) if os.path.exists(pkl_parameters) else read_parameters()
parameters.info()

# Profile report
profile = ProfileReport(parameters, title="Profiling Parameters")
profile.to_widgets()

profile.to_file(rep_path + 'parameters.html')
profile.to_file(rep_path + 'parameters.json')

# Display
parameters = parameters[['displayName','name','preferredUnit']]
display(parameters)

os.makedirs(clean_path, exist_ok=True)
parameters.to_json(clean_path + "parameters.json", orient='records')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 1 to 19843
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   description    22 non-null     object 
 1   displayName    22 non-null     object 
 2   isCore         22 non-null     bool   
 3   maxColorValue  11 non-null     float64
 4   name           22 non-null     object 
 5   preferredUnit  22 non-null     object 
dtypes: bool(1), float64(1), object(4)
memory usage: 1.1+ KB


,displayName,name,preferredUnit
id,,,
1,PM10,pm10,µg/m³
2,PM2.5,pm25,µg/m³
3,O₃ mass,o3,µg/m³
4,CO mass,co,µg/m³
5,NO₂ mass,no2,µg/m³
6,SO₂ mass,so2,µg/m³
7,NO₂,no2,ppm
8,CO,co,ppm
9,SO₂,so2,ppm


In [3]:
def read_countries():
    _json = json.load(open(rel_path + 'countries.json'))
    data = pd.DataFrame(_json['results'])
    data.to_pickle(pkl_countries, compression='infer', protocol=-1)
    return data

countries = pkl.load(open(pkl_countries, 'rb')) if os.path.exists(pkl_countries) else read_countries()
countries.info()

# Profile report
profile = ProfileReport(countries.loc[:, countries.columns != 'count'], title="Profiling Countries") # cannot insert count, already exists
profile.to_widgets()

profile.to_file(rep_path + 'countries.html')
profile.to_file(rep_path + 'countries.json')

# Display
countries = countries[['cities','code','name','parameters']]
display(countries)

countries.to_json(clean_path + "countries.json", orient='records')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cities        130 non-null    int64 
 1   code          130 non-null    object
 2   count         130 non-null    int64 
 3   firstUpdated  130 non-null    object
 4   lastUpdated   130 non-null    object
 5   locations     130 non-null    int64 
 6   name          130 non-null    object
 7   parameters    130 non-null    object
 8   sources       130 non-null    int64 
dtypes: int64(4), object(5)
memory usage: 9.3+ KB


/home/pc/anaconda3/envs/BasicDataScience/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,cities,code,name,parameters
0,2,AD,Andorra,"[co, no2, o3, pm10, so2]"
1,3,AE,United Arab Emirates,"[o3, pm1, pm10, pm25, um010, um025, um100]"
2,2,AF,Afghanistan,[pm25]
3,0,AM,Armenia,"[pm1, pm10, pm25, um010, um025, um100]"
4,0,AO,Angola,"[pm1, pm10, pm25, um010, um025, um100]"
...,...,...,...,...
125,1,UZ,Uzbekistan,"[o3, pm25]"
126,1,VM,VM,[pm25]
127,2,VN,Vietnam,"[pm1, pm10, pm25, um010, um025, um100]"
128,8,XK,Kosovo,"[co, no2, o3, pm1, pm10, pm25, so2, um010, um0..."


In [6]:
def read_locations():
    _json = json.load(open(rel_path + 'locations_ES_Lleida.json'))
    data = pd.DataFrame(_json['results'])
    data.to_pickle(pkl_locations, compression='infer', protocol=-1)
    return data

locations = pkl.load(open(pkl_locations, 'rb')) if os.path.exists(pkl_locations) else read_locations()
locations.info()

# Profile report
profile = ProfileReport(locations, title="Profiling Measurements")
profile.to_widgets()

profile.to_file(rep_path + 'locations.html')
profile.to_file(rep_path + 'locations.json')

locations = locations[['city','country','measurements','name','parameters']]
locations.to_json(clean_path + "locations.json", orient='records')
display(locations)
print(locations.name.values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   city          7 non-null      object
 1   coordinates   7 non-null      object
 2   country       7 non-null      object
 3   entity        7 non-null      object
 4   firstUpdated  7 non-null      object
 5   id            7 non-null      int64 
 6   isAnalysis    7 non-null      bool  
 7   isMobile      7 non-null      bool  
 8   lastUpdated   7 non-null      object
 9   measurements  7 non-null      int64 
 10  name          7 non-null      object
 11  parameters    7 non-null      object
 12  sensorType    7 non-null      object
 13  sources       7 non-null      object
dtypes: bool(2), int64(2), object(10)
memory usage: 814.0+ bytes


,city,country,measurements,name,parameters
0,Lleida,ES,87477,ES1348A,"[{'average': 6.12817871775504, 'count': 43689,..."
1,Lleida,ES,158200,ES1225A,"[{'average': 21.836622590619, 'count': 30402, ..."
2,Lleida,ES,44096,ES1588A,"[{'average': 31.143459724238, 'count': 44096, ..."
3,Lleida,ES,166789,ES1982A,"[{'average': -0.809283896764813, 'count': 4126..."
4,Lleida,ES,87526,ES2034A,"[{'average': 33.5927029547574, 'count': 43963,..."
5,Lleida,ES,141226,ES0014R,"[{'average': 52.9704737236434, 'count': 46715,..."
6,Lleida,ES,43718,ES1248A,"[{'average': 31.86282538085, 'count': 43718, '..."


['ES1348A' 'ES1225A' 'ES1588A' 'ES1982A' 'ES2034A' 'ES0014R' 'ES1248A']


In [5]:
def read_measurements():
    _json = json.load(open(rel_path + 'measurements_day_0.json'))
    data = pd.DataFrame(_json['results'])
    data.to_pickle(pkl_measurements, compression='infer', protocol=-1)
    return data

measurements = pkl.load(open(pkl_measurements, 'rb')) if os.path.exists(pkl_measurements) else read_measurements()
measurements.info()

# Profile report
profile = ProfileReport(measurements, title="Profiling Measurements")
profile.to_widgets()


profile.to_file(rep_path + 'measurements_day_0.html')
profile.to_file(rep_path + 'measurements_day_0.json')

# Display
measurements = measurements[['average', 'day', 'parameter', 'unit']]
pd.to_datetime(measurements['day'], format='%Y-%m-%d')

measurements.to_json(clean_path + "measurements.json", orient='records')
display(measurements)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2182 entries, 0 to 2181
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   average            2182 non-null   float64
 1   day                2182 non-null   object 
 2   displayName        2182 non-null   object 
 3   id                 2182 non-null   int64  
 4   measurement_count  2182 non-null   int64  
 5   name               2182 non-null   object 
 6   parameter          2182 non-null   object 
 7   parameterId        2182 non-null   int64  
 8   subtitle           2182 non-null   object 
 9   unit               2182 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 170.6+ KB


,average,day,parameter,unit
0,52.0092,2020-12-31,o3,µg/m³
1,11.5678,2020-12-31,pm10,µg/m³
2,14.7604,2020-12-31,no2,µg/m³
3,3.4066,2020-12-31,so2,µg/m³
4,322.8080,2020-12-31,co,µg/m³
...,...,...,...,...
2177,3.1374,2020-01-01,so2,µg/m³
2178,25.6587,2020-01-01,pm10,µg/m³
2179,412.7953,2020-01-01,co,µg/m³
2180,22.7752,2020-01-01,no2,µg/m³
